In [3]:
import pandas as pd

## Baixar dados de 2024

In [60]:
exportacoes_mundo_2024 = pd.read_excel('./dados/comex/Exportacao Mundo 2024 NCM.xlsx',dtype={'Código NCM':str})


In [61]:
exportacoes_mundo_2024.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,44963878520
1,12019000,"Soja, mesmo triturada, exceto para semeadura",42941894470


In [62]:
exportacao_eua_2024 = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx',dtype={'Código NCM':str})

In [63]:
exportacao_eua_2024.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,5830983860
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802


In [64]:
exportacoes_mundo_2024 = exportacoes_mundo_2024.rename(columns={'2024 - Valor US$ FOB': 'Valor US$ FOB Mundo'}).merge(exportacao_eua_2024.rename(columns={'2024 - Valor US$ FOB': 'Valor US$ FOB EUA'}).drop('Descrição NCM', axis=1), how='left', on='Código NCM').fillna(0)

In [65]:
exportacoes_mundo_2024['participacao_eua'] = exportacoes_mundo_2024['Valor US$ FOB EUA'] / exportacoes_mundo_2024['Valor US$ FOB Mundo']

In [66]:
exportacoes_mundo_2024.tail()

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua
8014,29161511,Oleato de manitol,0,0.0,NaN
8015,29329992,"1,3,4,6,7,8-Hexaidro-4,6,6,7,8,8-hexametilcicl...",0,0.0,NaN
8016,33012913,"Óleo essencial, de pau-santo (Bulnesia sarmien...",0,0.0,NaN
8017,51099000,"Outros fios de lã ou de pelos finos, acondicio...",0,0.0,NaN
8018,30021236,Soroalbumina humana,0,0.0,NaN


## Mapear NCM's para produtos

In [67]:
tradutor_ncm = pd.read_excel('./dados/mip/TradutorNCM.xls', dtype={'Código NCM': str, 'Código Produto': str})

In [68]:
exportacoes_mapeadas_scn = exportacoes_mundo_2024.merge(tradutor_ncm.drop('Nome NCM',axis=1), how='left', left_on='Código NCM', right_on='Código NCM', suffixes=('', '_tradutor'), validate='1:1',indicator=True)

In [69]:
exportacoes_nao_mapeadas = exportacoes_mapeadas_scn.query('_merge == "left_only"')

In [70]:
lista_posicao_nao_mapeadas = exportacoes_nao_mapeadas['Código NCM'].str[0:4].drop_duplicates().to_list()

In [71]:
lista_posicao_nao_mapeadas

['0207', '8504', '3906', '3207', '7315', '3824', '4811', '2710', '0305']

In [72]:
exportacoes_mapeadas_scn['Posicao NCM'] = exportacoes_mapeadas_scn['Código NCM'].str[0:4]


In [73]:
exportacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().sort_values(by='Posicao NCM')

,Posicao NCM,Código Produto,Descrição Produto
11,0207,10913,Carne de aves
762,0305,10914,Pescado industrializado
8,2710,19914,Óleo combustível
1838,2710,20922,Produtos químicos diversos
538,2710,19913,Naftas para petroquímica
35,2710,19916,Outros produtos do refino do petróleo
23,2710,19911,Combustíveis para aviação
685,3207,20923,"Tintas, vernizes, esmaltes e lacas"
440,3824,20922,Produtos químicos diversos
2541,3824,23002,"Artefatos de cimento, gesso e semelhantes"


In [74]:
substituicao_valores_nulos = exportacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` not in ["3824","7315","2710"]')

In [75]:
exportacoes_mapeadas_scn.query("_merge == 'left_only'").query('`Posicao NCM` == "3824"').head()

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,_merge,Posicao NCM
2965,38249961,Contrastes para exames de diagnóstico por imag...,942158,18782.0,0.019935,NaN,NaN,left_only,3824
6823,38249969,Outros contrastes para exames de diagnóstico p...,4277,0.0,0.000000,NaN,NaN,left_only,3824


In [77]:
substituicao_valores_nulos = pd.concat([substituicao_valores_nulos, exportacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` == "3824" and `Código Produto` == "20922"')])

In [78]:
exportacoes_mapeadas_scn.query("_merge == 'left_only'").query('`Posicao NCM` == "7315"').head()

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,_merge,Posicao NCM
2903,73151190,Outras correntes de rolos,1018373,29419.0,0.028888,NaN,NaN,left_only,7315


In [79]:
substituicao_valores_nulos = pd.concat([substituicao_valores_nulos, exportacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` == "7315" and `Código Produto` == "25001"')])

In [80]:
exportacoes_mapeadas_scn.query("_merge == 'left_only'").query('`Posicao NCM` == "2710"').head()


,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,_merge,Posicao NCM
7737,27109120,Outros resíduos de óleos que contenham terfeni...,123,0.0,0.0,NaN,NaN,left_only,2710


In [81]:
substituicao_valores_nulos = pd.concat([substituicao_valores_nulos, exportacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` == "2710" and `Código Produto` == "19916"')])

In [82]:
lista_posicao_nao_mapeadas

['0207', '8504', '3906', '3207', '7315', '3824', '4811', '2710', '0305']

In [83]:
substituicao_valores_nulos

,Posicao NCM,Código Produto,Descrição Produto
11,0207,10913,Carne de aves
65,8504,27001,"Máquinas, aparelhos e materiais elétricos"
145,4811,17002,"Papel, papelão, embalagens e artefatos de papel"
486,3906,20914,"Resinas,elastômeros e fibras artif. e sintéticas"
685,3207,20923,"Tintas, vernizes, esmaltes e lacas"
762,0305,10914,Pescado industrializado
440,3824,20922,Produtos químicos diversos
1152,7315,25001,"Produtos de metal, excl. máquinas e equipamentos"
35,2710,19916,Outros produtos do refino do petróleo


In [100]:
#substituir apenas os valores nulos de acordo com a tabela de substituição e manter os demais valores sem alteração

# Criar o mapeamento para ambas as colunas
mapeamento_codigo = substituicao_valores_nulos.set_index('Posicao NCM')['Código Produto']
mapeamento_descricao = substituicao_valores_nulos.set_index('Posicao NCM')['Descrição Produto']

# Substituir ambas onde é nulo E está na lista
mask = (exportacoes_mapeadas_scn['Descrição Produto'].isna()) & (exportacoes_mapeadas_scn['Posicao NCM'].isin(lista_posicao_nao_mapeadas))

exportacoes_mapeadas_scn.loc[mask, 'Código Produto'] = exportacoes_mapeadas_scn.loc[mask, 'Posicao NCM'].map(mapeamento_codigo)
exportacoes_mapeadas_scn.loc[mask, 'Descrição Produto'] = exportacoes_mapeadas_scn.loc[mask, 'Posicao NCM'].map(mapeamento_descricao)

In [104]:
#Verifica se a substituição foi feita corretamente
exportacoes_mapeadas_scn.query('_merge == "left_only"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM']].drop_duplicates()

,Posicao NCM
38,0207
848,8504
2702,3906
2877,3207
2903,7315
2965,3824
4816,4811
7737,2710
7765,0305


In [105]:
exportacoes_mapeadas_scn.drop(columns=['_merge'], inplace=True)

In [107]:
exportacoes_mapeadas_scn.head(2)

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,Posicao NCM
0,27090010,Óleos brutos de petróleo,44963878520,5.830984e+09,0.129682,06801,"Petróleo, gás natural e serviços de apoio",2709
1,12019000,"Soja, mesmo triturada, exceto para semeadura",42941894470,5.748195e+07,0.001339,01915,Soja em grão,1201


In [109]:
#Verificar a participação de cada NCM em relação ao total de exportações de um mesmo código de produto, considerando que todos os códigos de produto são válidos
exportacoes_mapeadas_scn['Total Exportações Produto Mundo'] = exportacoes_mapeadas_scn.groupby('Código Produto')['Valor US$ FOB Mundo'].transform('sum')
exportacoes_mapeadas_scn['Total Exportações Produto EUA'] = exportacoes_mapeadas_scn.groupby('Código Produto')['Valor US$ FOB EUA'].transform('sum')



In [113]:
exportacoes_mapeadas_scn['Participação Produto EUA'] = exportacoes_mapeadas_scn['Total Exportações Produto EUA'] / exportacoes_mapeadas_scn['Total Exportações Produto Mundo']

In [117]:
exportacoes_mapeadas_scn.query('`Código Produto` == "06801"')['Valor US$ FOB EUA'].sum()

np.float64(5830985624.0)

In [115]:
exportacoes_mapeadas_scn.query('`Código Produto` == "06801"')

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,Posicao NCM,Total Exportações Produto Mundo,Total Exportações Produto EUA,Participação Produto EUA
0,27090010,Óleos brutos de petróleo,44963878520,5.830984e+09,0.129682,06801,"Petróleo, gás natural e serviços de apoio",2709,44964297440,5.830986e+09,0.12968
3691,27090090,Óleos brutos de minerais betuminosos,406710,0.000000e+00,0.000000,06801,"Petróleo, gás natural e serviços de apoio",2709,44964297440,5.830986e+09,0.12968
6748,27111100,Gás natural liquefeito,5139,1.100000e+03,0.214049,06801,"Petróleo, gás natural e serviços de apoio",2711,44964297440,5.830986e+09,0.12968
6804,27141000,Xistos e areias betuminosos,4483,6.640000e+02,0.148115,06801,"Petróleo, gás natural e serviços de apoio",2714,44964297440,5.830986e+09,0.12968
7009,27112100,Gás natural no estado gasoso,2588,0.000000e+00,0.000000,06801,"Petróleo, gás natural e serviços de apoio",2711,44964297440,5.830986e+09,0.12968


In [114]:
exportacoes_mapeadas_scn.head()

,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,Posicao NCM,Total Exportações Produto Mundo,Total Exportações Produto EUA,Participação Produto EUA
0,27090010,Óleos brutos de petróleo,44963878520,5.830984e+09,0.129682,06801,"Petróleo, gás natural e serviços de apoio",2709,44964297440,5.830986e+09,0.129680
1,12019000,"Soja, mesmo triturada, exceto para semeadura",42941894470,5.748195e+07,0.001339,01915,Soja em grão,1201,42949763522,5.748327e+07,0.001338
2,26011100,"Minérios de ferro e seus concentrados, exceto ...",26574698749,5.410724e+06,0.000204,07911,Minério de ferro,2601,29860185978,3.924658e+08,0.013143
3,17011400,Outros açúcares de cana,15925320657,4.396613e+08,0.027608,10921,Açúcar,1701,18601830896,6.039729e+08,0.032468
4,09011110,"Café não torrado, não descafeinado, em grão",11331933183,1.895591e+09,0.167279,01918,Café em grão,0901,11338069646,1.896020e+09,0.167226


,Código NCM,Descrição NCM,Valor US$ FOB Mundo,Valor US$ FOB EUA,participacao_eua,Código Produto,Descrição Produto,Posicao NCM,Total Exportações Produto Mundo,Total Exportações Produto EUA
0,27090010,Óleos brutos de petróleo,44963878520,5.830984e+09,0.129682,06801,"Petróleo, gás natural e serviços de apoio",2709,44964297440,5.830986e+09
1,12019000,"Soja, mesmo triturada, exceto para semeadura",42941894470,5.748195e+07,0.001339,01915,Soja em grão,1201,42949763522,5.748327e+07
2,26011100,"Minérios de ferro e seus concentrados, exceto ...",26574698749,5.410724e+06,0.000204,07911,Minério de ferro,2601,29860185978,3.924658e+08
3,17011400,Outros açúcares de cana,15925320657,4.396613e+08,0.027608,10921,Açúcar,1701,18601830896,6.039729e+08
4,09011110,"Café não torrado, não descafeinado, em grão",11331933183,1.895591e+09,0.167279,01918,Café em grão,0901,11338069646,1.896020e+09


In [ ]:
exportacoes_mapeadas_scn.tail()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Código Produto,Descrição Produto,Posicao NCM,Total Importações Produto,Participação NCM no Produto
8743,94011010,"Assentos ejetáveis, dos tipos utilizados em ve...",5,30001,"Aeronaves, embarcações e outros equipamentos d...",9401,1.447465e+10,3.454314e-10
8744,03091000,"Farinhas, pós e ""pellets"" de peixes, para alim...",4,10914,Pescado industrializado,0309,7.377429e+08,5.421943e-09
8745,28121400,Pentacloreto de fósforo,3,20911,Produtos químicos inorgânicos,2812,1.435101e+10,2.090445e-10
8746,49070010,Papéis-moeda,2,18001,Serviços de impressão e reprodução,4907,5.012540e+07,3.989993e-08
8747,29242945,Iodamida,1,20913,Produtos químicos orgânicos,2924,1.236260e+10,8.088915e-11


In [ ]:
exportacoes_mapeadas_scn.to_excel('./Outputs/Importação Mundo 2024 NCM Mapeada SCN.xlsx', index=False)

## Selecionar pleitos CAT

In [46]:
pleitos = pd.read_excel('./dados/comex/pleitos_consolidados-25-07-25.xlsx', sheet_name='LETEC',skiprows=1)

In [47]:
pleitos_selecionados = pleitos.query('`Ex-tarifário` == "Não" and `Quota Pretendida`.isnull()')

In [52]:
pleitos_selecionados.loc[:,'alteracao_tarifaria'] = (pleitos_selecionados['Alíquota Pretendida'] )/ pleitos_selecionados['TEC Aplicada']

In [58]:
pleitos_selecionados

,Processo SEI Público,Link Processo SEI Público,Processo SEI Restrito,Pleiteante,Data Início Prazo de Manifestação,Data Término Prazo de Manifestação,Data de Término da Medida em Vigor,Efeito Tarifário Pretendido,Tipo do Pleito,NCM,...,Unidade da Quota,Prazo Pretendido (meses),Setor do Produto,Etapa do Pleito,Situação do Pleito,Avaliação RFB Necessária,Obs,Data de Apresentação no CAT,Data de Apresentação no Gecex,alteracao_tarifaria
33,19971.000522/2025-19,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.000523/2025-63,Basf S.A.,2025-06-05,2025-07-20,NaT,Elevação,Alteração,3906.90.54,...,NaN,NaN,Fabricação de produtos químicos,Strat,Em análise,NaN,Produto objeto do pleito teve a NCM alterada d...,NaT,NaT,inf
44,19971.000881/2025-76,NaN,19971.000882/2025-11,FAESP - Federação da Agricultura e Pecuária do...,2025-07-25,2025-09-08,2025-08-28,Elevação,Renovação,4001.29.20,...,NaN,12.0,Silvicultura e exploração madeireira,Prazo para manifestações públicas,NaN,NaN,NaN,NaT,NaT,3.055556
46,19971.000883/2025-65,NaN,19971.000884/2025-18,FAESP - Federação da Agricultura e Pecuária do...,2025-07-25,2025-09-08,2025-08-28,Elevação,Renovação,4001.22.00,...,NaN,12.0,"Produção vegetal, animal e caça",Prazo para manifestações públicas,NaN,NaN,NaN,NaT,NaT,3.055556
49,19971.000258/2025-13,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.000259/2025-68,"Emteco Comércio e Serviços em automação, motor...",2025-07-25,2025-09-08,NaT,Elevação,Inclusão,8501.40.19,...,NaN,NaN,Fabricação de equipamentos elétricos,Prazo para manifestações públicas,NaN,NaN,Migrado de LEBITBK,NaT,NaT,2.160494
48,19971.000260/2025-92,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.000261/2025-37,"Emteco Comércio e Serviços em automação, motor...",2025-07-25,2025-09-08,NaT,Elevação,Inclusão,8501.31.10,...,NaN,NaN,Fabricação de equipamentos elétricos,Prazo para manifestações públicas,NaN,NaN,Migrado de LEBITBK,NaT,NaT,1.944444
36,19971.000640/2025-27,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.000641/2025-71,Basf S.A.,2025-06-16,2025-07-31,NaT,Elevação,Inclusão,2916.12.30,...,NaN,NaN,Fabricação de produtos químicos,Prazo para manifestações públicas,NaN,NaN,NaN,NaT,NaT,1.851852
30,19971.000586/2025-10,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.000587/2025-64,Basf S.A.,2025-06-05,2025-07-20,2025-10-14,Elevação,Renovação,3908.10.26,...,NaN,NaN,Fabricação de produtos químicos,Strat,Em análise,NaN,NaN,NaT,NaT,1.587302
3,19971.002083/2024-06,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.002084/2024-42,ABRAPLA - Associação Brasileita da Indústria d...,2024-10-25,2024-12-09,NaT,Elevação,Inclusão,3921.90.19,...,NaN,12.0,Fabricação de produtos de borracha e plásticos,CAT,Em análise,NaN,NaN,2025-07-25,NaT,1.562500
4,19971.002081/2024-17,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.002082/2024-53,ABRAPLA - Associação Brasileita da Indústria d...,2024-10-25,2024-12-09,NaT,Elevação,Inclusão,3919.90.20,...,NaN,12.0,Fabricação de produtos de borracha e plásticos,CAT,Em análise,NaN,NaN,2025-07-25,NaT,1.562500
7,19971.002111/2024-87,https://colaboragov.sei.gov.br/sei/processo_ac...,19971.002112/2024-21,ABRAPLA - Associação Brasileita da Indústria d...,2024-11-05,2024-12-20,NaT,Elevação,Inclusão,3920.43.90,...,NaN,NaN,Fabricação de produtos de borracha e plásticos,Gecex,Em análise,NaN,NaN,2025-06-27,2025-08-28,1.562500


## Verificar impacto nas importações


In [79]:
pleito_selecionado = pleitos_selecionados.iloc[1,:]

ncm_pleito_selecionado = pleito_selecionado['NCM'].replace('.','')
alteracao_tarifaria_pleito_selecionado = pleito_selecionado['alteracao_tarifaria']


In [ ]:
participação_ncm_produto = exportacoes_mapeadas_scn.query('`Código NCM` == @ncm_pleito_selecionado')['Participação NCM no Produto']
produto_selecionado = exportacoes_mapeadas_scn.query('`Código NCM` == @ncm_pleito_selecionado')['Código Produto']

In [91]:
alteracao_imposto = 1+(participação_ncm_produto* alteracao_tarifaria_pleito_selecionado)

In [92]:
produto_selecionado

1801    02801
Name: Código Produto, dtype: object

In [156]:
alteracao_imposto

1801    1.162061
Name: Participação NCM no Produto, dtype: float64

In [189]:
matriz_distribuicao_importacao = pd.read_excel('./dados/mip/Matriz_Distribuicao_Importacao_2019_68x68MetodoGuilhoto.xlsx',header=None,skiprows=1)

In [190]:
imposto_importacao =pd.read_excel('./dados/mip/Oferta_Importacao_2019_68x68MetodoGuilhoto.xlsx',header=None,skiprows=1)

In [191]:
#Multiplicar a matriz de distribuição de importação pelo vetor com o imposto de importação para saber como os impostos são distribuídos entre os produtos e ter uma matriz com o resultado
imposto_ii_setor_original = matriz_distribuicao_importacao.mul(imposto_importacao.iloc[:,0], axis=0).sum()


In [192]:
imposto_importacao.iloc[13,0] = imposto_importacao.iloc[13,0]*alteracao_imposto

/tmp/ipykernel_761231/2485346180.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '59.2651311940688' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  imposto_importacao.iloc[13,0] = imposto_importacao.iloc[13,0]*alteracao_imposto


In [193]:
matriz_distribuicao_importacao.shape

(128, 74)

In [195]:
imposto_ii_setor_alteracao_tarifaria = matriz_distribuicao_importacao.mul(imposto_importacao.iloc[:,0], axis=0).sum()

In [196]:
imposto_ii_setor_alteracao_tarifaria.shape

(74,)

In [197]:
imposto_ii_setores_consumo_intermediario = imposto_ii_setor_alteracao_tarifaria.iloc[0:68]

In [198]:
imposto_ii_setores_consumo_intermediario.shape

(68,)

In [200]:
imposto_ii_setores_consumo_intermediario

0     766.159033
1     206.576468
2      15.119476
3      81.304173
4     619.418696
         ...    
63    296.612649
64    759.320789
65     19.259619
66    165.018972
67      0.000000
Length: 68, dtype: float64

In [203]:
vetor_alteração_imposto = imposto_ii_setores_consumo_intermediario/imposto_ii_setor_original.iloc[0:68]

## Calcular a matriz de coeficientes técnicos e demais componentes do modelo de preços

In [124]:
matriz_guilhoto =pd.read_excel('./Outputs/MIP_2019_68x68MetodoGuilhoto.xlsx')

In [127]:
matriz_guilhoto.tail()

,Setores,"Agricultura, inclusive o apoio à agricultura","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não-,"Extração de petróleo e gás, inclusive as ativ","Extração de minério de ferro, inclusive benef","Extração de minerais metálicos não-ferrosos,","Abate e produtos de carne, inclusive os produ",Fabricação e refino de açúcar,...,Serviços domésticos,Consumo Intermediário,Exportação,Consumo do Governo,Consumo das ISFLSF,Consumo das Famílias,FBCF,Variação do estoque,Demanda Final,Demanda Total
84,Excedente operacional bruto (EOB),80576.0,13240.0,12652.0,3785.0,96116.0,46550.0,4713.0,12186.0,-1235.0,...,0,2430341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2430341.0
85,Outros impostos sobre a produção,1990.0,688.0,177.0,200.0,1729.0,723.0,180.0,2938.0,1032.0,...,0,102282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102282.0
86,Outros subsídios à produção,-7520.0,-78.0,-31.0,-38.0,0.0,-15.0,0.0,-50.0,-71.0,...,0,-11211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-11211.0
87,Valor da produção,415308.0,163008.0,38200.0,20360.0,254621.0,108090.0,19375.0,322433.0,52577.0,...,75158,12741791.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12741791.0
88,Fator trabalho (ocupações),6510033.0,5830589.0,847266.0,111278.0,52669.0,25542.0,29890.0,784540.0,136690.0,...,6843869,105995759.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105995759.0


In [151]:
valor_producao = matriz_guilhoto.iloc[87,1:69]

In [252]:
consumo_intermediario = matriz_guilhoto.iloc[:68,1:69]


In [253]:
#matriz_guilhoto.iloc[69:74,0:69]
importacao_modelo_precos = matriz_guilhoto.iloc[69,1:69]
impostos_modelo_precos = matriz_guilhoto.iloc[70:74,1:69]

valor_agregado = matriz_guilhoto.iloc[75,1:69]

In [254]:
coeficientes_tecnicos = consumo_intermediario.div(valor_producao, axis=1)

In [226]:
#dividir cada linha da matriz de impostos_modelo_precos pelo valor de produção, de forma que o elemento de cada linha seja dividido pelo valor de produção, na coluna correspondente
importacao_modelo_precos_part = importacao_modelo_precos.div(valor_producao, axis=0)
valor_agregado_part = valor_agregado.div(valor_producao, axis=0)
# dividir cada linha da matriz de impostos_modelo_precos pelo valor de produção, de forma que o elemento de cada linha seja dividido pelo valor de produção, na coluna correspondente
impostos_modelo_precos_part = impostos_modelo_precos.div(valor_producao, axis=1)


In [255]:
coeficientes_tecnicos.head()

,"Agricultura, inclusive o apoio à agricultura","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não-,"Extração de petróleo e gás, inclusive as ativ","Extração de minério de ferro, inclusive benef","Extração de minerais metálicos não-ferrosos,","Abate e produtos de carne, inclusive os produ",Fabricação e refino de açúcar,Outros produtos alimentares,...,Outras atividades administrativas e serviços,"Atividades de vigilância, segurança e investi","Administração pública, defesa e seguridade so",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetác",Organizações associativas e outros serviços p,Serviços domésticos
0,0.021052,0.033799,0.004641,0.000082,0.000036,0.000083,0.000092,0.024467,0.54847,0.236871,...,0.000561,0.000005,0.000958,0.000956,0.000372,0.00193,0.000704,0.000058,0.001783,0.0
1,0.002674,0.041338,0.005611,0.000209,0.00009,0.000152,0.000169,0.286305,0.003,0.014076,...,0.000127,0.000008,0.000329,0.000337,0.000124,0.000597,0.000359,0.000075,0.000237,0.0
2,0.003738,0.009273,0.062117,0.000089,0.000007,0.000011,0.000013,0.006069,0.000012,0.000772,...,0.000009,0.000001,0.000099,0.000189,0.00006,0.000362,0.00006,0.000007,0.000018,0.0
3,0.000369,0.002805,0.000376,0.013853,0.002906,0.000012,0.000084,0.000197,0.000174,0.000941,...,0.000009,0.000003,0.000083,0.000049,0.00001,0.000042,0.000014,0.000027,0.000016,0.0
4,0.000047,0.000089,0.000023,0.000137,0.022671,0.005211,0.00222,0.000221,0.000008,0.001832,...,0.000035,0.000016,0.002299,0.000055,0.000085,0.000007,0.000022,0.000235,0.000154,0.0


In [259]:
#concatenar os 4 vetores em um único DataFrame
matriz_modelo_precos = pd.concat([coeficientes_tecnicos,pd.DataFrame(importacao_modelo_precos_part.values.reshape(1,68),columns=coeficientes_tecnicos.columns),impostos_modelo_precos_part,pd.DataFrame(valor_agregado_part.values.reshape(1,68),columns=coeficientes_tecnicos.columns)], axis=0,ignore_index=True)


In [261]:
coeficientes_tecnicos

,"Agricultura, inclusive o apoio à agricultura","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não-,"Extração de petróleo e gás, inclusive as ativ","Extração de minério de ferro, inclusive benef","Extração de minerais metálicos não-ferrosos,","Abate e produtos de carne, inclusive os produ",Fabricação e refino de açúcar,Outros produtos alimentares,...,Outras atividades administrativas e serviços,"Atividades de vigilância, segurança e investi","Administração pública, defesa e seguridade so",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetác",Organizações associativas e outros serviços p,Serviços domésticos
0,0.021052,0.033799,0.004641,0.000082,0.000036,0.000083,0.000092,0.024467,0.54847,0.236871,...,0.000561,0.000005,0.000958,0.000956,0.000372,0.00193,0.000704,0.000058,0.001783,0.0
1,0.002674,0.041338,0.005611,0.000209,0.00009,0.000152,0.000169,0.286305,0.003,0.014076,...,0.000127,0.000008,0.000329,0.000337,0.000124,0.000597,0.000359,0.000075,0.000237,0.0
2,0.003738,0.009273,0.062117,0.000089,0.000007,0.000011,0.000013,0.006069,0.000012,0.000772,...,0.000009,0.000001,0.000099,0.000189,0.00006,0.000362,0.00006,0.000007,0.000018,0.0
3,0.000369,0.002805,0.000376,0.013853,0.002906,0.000012,0.000084,0.000197,0.000174,0.000941,...,0.000009,0.000003,0.000083,0.000049,0.00001,0.000042,0.000014,0.000027,0.000016,0.0
4,0.000047,0.000089,0.000023,0.000137,0.022671,0.005211,0.00222,0.000221,0.000008,0.001832,...,0.000035,0.000016,0.002299,0.000055,0.000085,0.000007,0.000022,0.000235,0.000154,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.0,0.0,0.0,0.000003,0.0,0.000048,0.0,0.000009,0.000006,0.000004,...,0.000001,0.0,0.000001,0.0,0.000001,0.000001,0.001109,0.000001,0.0,0.0
64,0.0,0.0,0.0,0.000002,0.000003,0.000002,0.000002,0.000004,0.000005,0.000003,...,0.000022,0.00001,0.000082,0.00002,0.000055,0.000057,0.121792,0.000151,0.000035,0.0
65,0.000006,0.000088,0.000087,0.000062,0.000101,0.000008,0.000006,0.000284,0.000121,0.000379,...,0.00031,0.000153,0.000856,0.000248,0.000628,0.000397,0.000019,0.018753,0.006056,0.0
66,0.000384,0.000265,0.000673,0.00229,0.00378,0.00417,0.003209,0.001005,0.001716,0.000925,...,0.005933,0.001048,0.00086,0.000357,0.004211,0.0075,0.009069,0.00144,0.003935,0.0


In [268]:
#Calcular a inversa de leontief, que é igual a (I - transposta(coeficientes_tecnicos))^-1

import numpy as np
from scipy.linalg import inv
I = np.eye(coeficientes_tecnicos.shape[1])
inversa_leontief = inv(I - coeficientes_tecnicos.values.T.astype(float))




In [270]:
matriz_modelo_precos = pd.concat([pd.DataFrame(importacao_modelo_precos_part.values.reshape(1,68),columns=coeficientes_tecnicos.columns),impostos_modelo_precos_part,pd.DataFrame(valor_agregado_part.values.reshape(1,68),columns=coeficientes_tecnicos.columns)], axis=0,ignore_index=True)


In [274]:
vetor_alteração_imposto

0     1.000801
1     1.002867
2     1.051692
3     1.000007
4     1.000000
        ...   
63    1.000000
64    1.000000
65    1.000000
66    1.000000
67         NaN
Length: 68, dtype: float64

In [276]:
#multiplicar cada elento do vetor de alteração pela segunda linha da matriz de modelo de preços, que é a linha de imposto de importação
matriz_modelo_precos.iloc[1,:] = matriz_modelo_precos.iloc[1,:] * vetor_alteração_imposto.values

In [279]:
matriz_modelo_precos

,"Agricultura, inclusive o apoio à agricultura","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não-,"Extração de petróleo e gás, inclusive as ativ","Extração de minério de ferro, inclusive benef","Extração de minerais metálicos não-ferrosos,","Abate e produtos de carne, inclusive os produ",Fabricação e refino de açúcar,Outros produtos alimentares,...,Outras atividades administrativas e serviços,"Atividades de vigilância, segurança e investi","Administração pública, defesa e seguridade so",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetác",Organizações associativas e outros serviços p,Serviços domésticos
0,0.070669,0.039781,0.01675,0.078741,0.12309,0.069697,0.095315,0.023318,0.018415,0.054571,...,0.024375,0.0157,0.017814,0.012453,0.025294,0.037889,0.036946,0.030563,0.058264,0.0
1,0.001845,0.001267,0.000396,0.003993,0.002433,0.002556,0.002394,0.000813,0.000239,0.001668,...,0.000893,0.000356,0.000184,0.000342,0.000089,0.001367,0.002343,0.000414,0.000892,NaN
2,0.000204,0.000617,0.000224,0.001699,0.000682,0.001048,0.000951,0.001034,0.000194,0.001226,...,0.001284,0.000412,0.000158,0.00034,0.000212,0.000717,0.001411,0.000564,0.001208,0.0
3,0.021779,0.025957,0.01257,0.013408,0.008518,0.010515,0.011572,0.02438,0.005657,0.018394,...,0.013577,0.005925,0.006059,0.005712,0.007756,0.016614,0.024425,0.011052,0.019764,0.0
4,0.014677,0.011826,0.007458,0.022239,0.027591,0.018823,0.022455,0.017853,0.025075,0.015734,...,0.012272,0.009296,0.016158,0.005168,0.011331,0.010286,0.010516,0.017387,0.018249,0.0
5,0.496107,0.472934,0.72212,0.405403,0.456694,0.470645,0.382297,0.14679,0.124446,0.182431,...,0.668703,0.804412,0.720023,0.842481,0.704286,0.644113,0.559299,0.560718,0.50708,1.0


In [282]:
#somar todas as linhas coluna a coluna da matriz_modelo_precos
matriz_modelo_precos = matriz_modelo_precos.sum(axis=0)


In [283]:
#multiplicar a inversa de leontief pela matriz de modelo de preços
resultado_modelo_precos = inversa_leontief @ matriz_modelo_precos.values

In [290]:
lista_nomes = matriz_modelo_precos.index.values

In [298]:
lista_nomes.shape

(68,)

In [300]:
resultado_modelo_precos.reshape(1,68)

array([[np.float64(1.000001810070509), np.float64(1.0000043735253301),
        np.float64(1.0000208865399864), np.float64(1.0000002205398975),
        np.float64(1.0000002253861366), np.float64(1.0000001733633275),
        np.float64(1.000000172464513), np.float64(1.0000022953317163),
        np.float64(1.000001088831042), np.float64(1.0000011530273016),
        np.float64(1.0000004744558373), np.float64(1.000001339309468),
        np.float64(1.0000007683135859), np.float64(1.0000004051844225),
        np.float64(1.0000005577420088), np.float64(1.0000259192585617),
        np.float64(1.0000151326092204), np.float64(1.0000011512673248),
        np.float64(1.0000001949168664), np.float64(1.0000009503573002),
        np.float64(1.0000005899179145), np.float64(1.0000002759837405),
        np.float64(1.0000008149069706), np.float64(1.0000003038175374),
        np.float64(1.0000026758995277), np.float64(1.0000008303765024),
        np.float64(1.0000029380233135), np.float64(1.000000201856595

In [ ]:
pd.DataFrame(resultado_modelo_precos, index=lista_nomes, columns=['resultado']).sort_values(by='resultado', ascending=False).to_excel('./Outputs/Resultado Modelo Preços.xlsx', index=True)